In [1]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

conf = SparkConf()

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1617185507602_0003,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Cannot run multiple SparkContexts at once; existing SparkContext(app=livy-session-2, master=yarn) created by __init__ at /tmp/8206688282228336114:595 
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 133, in __init__
    SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 340, in _ensure_initialized
    callsite.function, callsite.file, callsite.linenum))
ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=livy-session-2, master=yarn) created by __init__ at /tmp/8206688282228336114:595 



In [2]:
play_count = spark.read.parquet('s3://17700project/play_counts.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
seed = 180000
(split_80_df, split_20_df) = play_count.randomSplit([0.9, 0.1], seed = seed)

# Let's cache these datasets for performance
training_df = split_80_df.cache()
test_df = split_20_df.cache()

print('Training: {0}, test: {1}\n'.format(
  training_df.count(), test_df.count())
)
training_df.show(3)
test_df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training: 43537939, test: 4835646

+------------------+--------------------+-----+----------+----------+
|            songId|              userId|Plays|new_userId|new_songId|
+------------------+--------------------+-----+----------+----------+
|SOAAEKX12A6D4F7E4E|00732e30a5abe4bec...|    1|    950928|    359777|
|SOAAEKX12A6D4F7E4E|00a3ee4941f7799ac...|    5|    473004|    359777|
|SOAAEKX12A6D4F7E4E|0129ee80d24fc9fe4...|    2|    508986|    359777|
+------------------+--------------------+-----+----------+----------+
only showing top 3 rows

+------------------+--------------------+-----+----------+----------+
|            songId|              userId|Plays|new_userId|new_songId|
+------------------+--------------------+-----+----------+----------+
|SOAAEKX12A6D4F7E4E|03bfb186b1c7fbe92...|    1|    385109|    359777|
|SOAAEKX12A6D4F7E4E|059266e024833d7a2...|    1|     67922|    359777|
|SOAAEKX12A6D4F7E4E|065c1954997224976...|    1|    227403|    359777|
+------------------+----------

In [15]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
import time 

strt_time = time.time()
# Let's initialize our ALS learner
als = ALS()

#df_pandas = sc.createDataFrame(training_df.head(50), columns=training_df.columns)
# Now set the parameters for the method
als.setMaxIter(5)\
   .setSeed(seed)\
   .setItemCol("new_songId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")
print('out')
# Now let's compute an evaluation metric for our test dataset
# We Create an RMSE evaluator using the label and predicted columns
reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")

tolerance = 0.03
ranks = [4, 8, 12, 16] #Tuning
regParams = [0.15, 0.2, 0.25] #Tuning
errors = [[0]*len(ranks)]*len(regParams)
models = [[0]*len(ranks)]*len(regParams)
err = 0
min_error = float('inf')
best_rank = -1
i = 0
for regParam in regParams:
    j = 0
    print('first')
    for rank in ranks:
        print('sec')
        # Set the rank here:
        # train = train.filter(train.new_userId < sys.maxsize).filter(train.new_songId < sys.maxsize)
        als.setParams(rank = rank, regParam = regParam)
        # Create the model with these parameters.
        model = als.fit(training_df)
        # Run the model to create a prediction. Predict against the validation_df.
        predict_df = model.transform(validation_df)

        # Remove NaN values from prediction (due to SPARK-14489)
        predicted_plays_df = predict_df.filter(predict_df.prediction != float('nan'))
        predicted_plays_df = predicted_plays_df.withColumn("prediction", F.abs(F.round(predicted_plays_df["prediction"],0)))
        # Run the previously created RMSE evaluator, reg_eval, on the predicted_ratings_df DataFrame
        error = reg_eval.evaluate(predicted_plays_df)
        errors[i][j] = error
        models[i][j] = model
        print('For rank %s, regularization parameter %s the RMSE is %s' % (rank, regParam, error))
        if error < min_error:
            min_error = error
            best_params = [i,j]
        j += 1
    i += 1

als.setRegParam(regParams[best_params[0]])
als.setRank(ranks[best_params[1]])
print('The best model was trained with regularization parameter %s' % regParams[best_params[0]])
print('The best model was trained with rank %s' % ranks[best_params[1]])
my_model = models[best_params[0]][best_params[1]]
end_time = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

out
first
sec
For rank 4, regularization parameter 0.15 the RMSE is 6.8525437557468685
sec
For rank 8, regularization parameter 0.15 the RMSE is 6.619838365852312
sec
For rank 12, regularization parameter 0.15 the RMSE is 6.5561938526724255
sec
For rank 16, regularization parameter 0.15 the RMSE is 6.457795264677563
first
sec
For rank 4, regularization parameter 0.2 the RMSE is 6.706337621866709
sec
For rank 8, regularization parameter 0.2 the RMSE is 6.48005677521155
sec
For rank 12, regularization parameter 0.2 the RMSE is 6.4367203988905555
sec
For rank 16, regularization parameter 0.2 the RMSE is 6.335132986443452
first
sec
For rank 4, regularization parameter 0.25 the RMSE is 6.609541665734989
sec
For rank 8, regularization parameter 0.25 the RMSE is 6.389694871225723
sec
For rank 12, regularization parameter 0.25 the RMSE is 6.3522922753247135
sec
For rank 16, regularization parameter 0.25 the RMSE is 6.247886762104598
The best model was trained with regularization parameter 0.25

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline

# Let's initialize our ALS learner
als = ALS()

#df_pandas = sc.createDataFrame(training_df.head(50), columns=training_df.columns)
# Now set the parameters for the method
als.setMaxIter(5)\
   .setSeed(seed)\
   .setItemCol("new_songId")\
   .setRatingCol("Plays")\
   .setUserCol("new_userId")

ALSPipeline = Pipeline()

ALSPipeline.setStages([als])

reg_eval = RegressionEvaluator(predictionCol="prediction", labelCol="Plays", metricName="rmse")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

crossval = CrossValidator(estimator = ALSPipeline, evaluator = reg_eval, numFolds=10)

ranks = [16]
regParams = [0.25]

paramGrid = (ParamGridBuilder()
             .addGrid(als.rank, ranks)
             .addGrid(als.regParam, regParams)
             .build())

crossval.setEstimatorParamMaps(paramGrid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

CrossValidator_3387a582137f

In [6]:
import time

strt_time = time.time()
cvModel = crossval.fit(training_df).bestModel
end_time = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
strt_time - end_time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-1637.989329814911

In [8]:
reg_eval.evaluate(cvModel.transform(test_df).na.drop())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6.8729582380117415

In [10]:
cvModel.transform(test_df).

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+-----+----------+----------+----------+
|            songId|              userId|Plays|new_userId|new_songId|prediction|
+------------------+--------------------+-----+----------+----------+----------+
|SOLWEWK12AB01868C7|6574b1ab665c4f7d8...|    1|     11897|       148| 2.6320627|
|SOLWEWK12AB01868C7|782a0aaa53974624e...|    1|     12273|       148| 1.7617823|
|SOLWEWK12AB01868C7|965b46681bf99de91...|    1|    768967|       148| 2.7085574|
|SOLWEWK12AB01868C7|40507bd795fd2126a...|    1|    198619|       148| 2.0548723|
|SOLWEWK12AB01868C7|30a7cfb4df2883de4...|    2|    474675|       148|  5.706163|
+------------------+--------------------+-----+----------+----------+----------+
only showing top 5 rows

In [9]:
pred = cvModel.transform(test_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
pred.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in pred.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+-----+----------+----------+----------+
|songId|userId|Plays|new_userId|new_songId|prediction|
+------+------+-----+----------+----------+----------+
|     0|     0|    0|         0|         0|      9089|
+------+------+-----+----------+----------+----------+

In [12]:
strt_time = time.time()
user_id = [[17]]
# convert this into a dataframe so that it can be passed into the recommendForUserSubset
df = sc.parallelize(user_id).toDF(['new_userId'])
u = cvModel.stages[0].recommendForUserSubset(df, 5).rdd.collect()
end_time = time.time()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
strt_time - end_time

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-32.22424364089966

In [11]:
u

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(new_userId=17, recommendations=[Row(new_songId=172782, rating=40.726383209228516), Row(new_songId=300785, rating=39.22613525390625), Row(new_songId=12525, rating=37.18057632446289), Row(new_songId=249444, rating=29.71829605102539), Row(new_songId=79105, rating=26.729434967041016)])]

In [19]:
dir(ALS())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_call_java', '_clear', '_copyValues', '_copy_params', '_create_from_java_class', '_create_model', '_create_params_from_java', '_defaultParamMap', '_dummy', '_empty_java_param_map', '_fit', '_fit_java', '_from_java', '_input_kwargs', '_java_obj', '_make_java_param_pair', '_new_java_array', '_new_java_obj', '_paramMap', '_params', '_randomUID', '_resetUid', '_resolveParam', '_set', '_setDefault', '_shouldOwn', '_to_java', '_transfer_param_map_from_java', '_transfer_param_map_to_java', '_transfer_params_from_java', '_transfer_params_to_java', 'alpha', 'checkpointInterval', 'coldStartStrategy', 'copy', 'ex